#Natural Language Processing (Character Generation) with Recurrent Neural Network


We will generate charater using a RNN.  We will work with a dataset of Shakespeare's writing.  Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

- The model is character-based.
- The model is trained on small batches of text (100 characters each), and is able to generate a longer sequence of text with coherent structure.

*This guide is based on the following: https://www.tensorflow.org/tutorials/text/text_generation*


In [1]:
# %tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

###Download the Shakespeare Dataset

Here, we use an extract from a Shadespheare play for training.  We can use our own text paragraph data and use it for network training.

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# If you load your own data file, use the following lines:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

1115394/1115394 [==============================] - 1s 1us/step


###Read Contents of File
Let's look at the contents of the file.

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


###Encode
Encode each unique character as a different integer.

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [7]:
# Chect how a part of the text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


Make a function that can convert the numeric values back to text.


In [8]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


###Create Training Examples

Given a character, or a sequence of characters, what is the most probable next character? This is the task we are training the model to perform.

Our task is to feed the model a sequence and have it return the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples. 

The training examples we prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

In [9]:
# Create a stream of characters from our text data
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Use the batch method to turn this stream of characters into batches of desired length
seq_length = 100  # length of sequence for a training example
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
# Use these sequences of length 101 and split them into input and output.
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [10]:
for x, y in dataset.take(3):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


EXAMPLE

INPUT
now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us k

OUTPUT
ow Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us ki


Make training batches.

In [11]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

###Build the Model
We use an embedding layer, a LSTM layer, and one dense layer that contains a node for each unique character in our training data.  The dense layer will give us a probability distribution over all nodes.

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    # tf.keras.layers.LSTM(rnn_units,
    #                     return_sequences=True,
    #                     stateful=True,
    #                     recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),                        
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


###Create a Loss Function
Our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.

We will create the loss function.

Before we do that, let's have a look at a sample input and the output from the untrained model. 

In [13]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [14]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-4.40546125e-03 -6.76314859e-03 -8.00384022e-03 ... -1.41191799e-02
   -9.46008973e-03  1.70969707e-03]
  [ 4.70358878e-04 -1.35234289e-03 -3.12279118e-03 ... -1.43951653e-02
    1.42149553e-02  1.48805638e-03]
  [-3.82129569e-03 -6.68097427e-03 -7.17407931e-03 ... -2.21873764e-02
    3.44623812e-04  1.22494658e-03]
  ...
  [-4.08652844e-03 -1.22589571e-02 -9.37549863e-04 ... -1.10286716e-02
   -3.46921338e-03 -4.25200537e-03]
  [-6.94021815e-04 -1.27471462e-02  2.05978425e-03 ... -9.71968938e-03
   -2.65241284e-02  5.73199894e-03]
  [-3.17081483e-03 -1.52327493e-02 -1.07558314e-02 ... -1.83389615e-02
   -2.26809997e-02  3.32279364e-03]]

 [[-3.96211725e-03 -7.04186270e-03  6.42947201e-03 ...  6.11573923e-04
   -1.05459448e-02  1.64858550e-02]
  [ 5.08618960e-03 -1.16264424e-03  2.12142393e-02 ...  1.05180200e-02
   -7.71649973e-03  9.18526389e-03]
  [ 1.11748036e-02 -8.25536437e-03  1.59478988e-02 ...  4.24748519e-03
   -1.01631414e-02 -1.78208575e-06]
  ...
  [ 1.168

In [15]:
# Let's examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00440546 -0.00676315 -0.00800384 ... -0.01411918 -0.00946009
   0.0017097 ]
 [ 0.00047036 -0.00135234 -0.00312279 ... -0.01439517  0.01421496
   0.00148806]
 [-0.0038213  -0.00668097 -0.00717408 ... -0.02218738  0.00034462
   0.00122495]
 ...
 [-0.00408653 -0.01225896 -0.00093755 ... -0.01102867 -0.00346921
  -0.00425201]
 [-0.00069402 -0.01274715  0.00205978 ... -0.00971969 -0.02652413
   0.005732  ]
 [-0.00317081 -0.01523275 -0.01075583 ... -0.01833896 -0.022681
   0.00332279]], shape=(100, 65), dtype=float32)


In [16]:
# and finally we look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-4.4054613e-03 -6.7631486e-03 -8.0038402e-03 -6.4665852e-03
  4.2965263e-03 -1.2235662e-03  2.3354322e-03 -8.1822844e-03
 -6.5563461e-03  1.2604610e-02  2.1996177e-03  5.1977737e-03
 -3.2851482e-03 -3.3443836e-03  7.1773706e-03 -1.5454153e-03
  1.3071117e-03 -1.2089167e-03  7.9362681e-03  4.1687884e-03
 -1.2066921e-02  5.0356672e-03 -1.7509906e-02  1.0522880e-02
 -6.2785656e-03  8.3569307e-03  2.9254495e-03  1.4099050e-03
 -4.0622479e-03 -7.0068045e-03  9.5247189e-03 -6.0132341e-03
  9.0967258e-04 -3.2445574e-03  2.3598662e-03  5.7003852e-03
 -2.5659150e-03  1.0925613e-02 -7.5771087e-03 -1.0033136e-02
  6.1738947e-03  1.7296253e-02 -4.2142714e-03 -8.8770743e-03
 -2.3000219e-03 -4.8617651e-03 -2.7093259e-03 -1.6771210e-03
 -3.1213975e-05 -1.0590399e-02  5.9370254e-03 -4.4274381e-03
  2.2570244e-03 -6.8536135e-03 -3.0322871e-03 -7.7514304e-03
 -2.7966150e-04 -3.3698610e-03  5.0836969e-03 -2.1894239e-02
  7.4656894e-03  4.4627171e-03 -1.4119180e-02 -9.4600897e-03
  1.709697

In [17]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

'utc\nwANwY LY;nqhb&OmjUMhIbhBrn:qr dWqKUBDZO$ERPPESzav ,AtGd.ocRF:p?&iNLjKDQnP rVq:BkWzT NBGXs,?b?seU'

So, we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were. 

In [18]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

###Compile the Model
We can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming next. 


In [19]:
model.compile(optimizer='adam', loss=loss)

###Create Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [20]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

###Train the Model
**If this is taking a while go to Runtime > Change Runtime Type and choose "GPU" under hardware accelerator.**

In [21]:
history = model.fit(data, epochs=30, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 17s 68ms/step - loss: 2.6559
Epoch 2/30
172/172 [==============================] - 12s 62ms/step - loss: 1.9573
Epoch 3/30
172/172 [==============================] - 13s 63ms/step - loss: 1.6904
Epoch 4/30
172/172 [==============================] - 13s 63ms/step - loss: 1.5416
Epoch 5/30
172/172 [==============================] - 12s 61ms/step - loss: 1.4536
Epoch 6/30
172/172 [==============================] - 12s 61ms/step - loss: 1.3939
Epoch 7/30
172/172 [==============================] - 13s 62ms/step - loss: 1.3474
Epoch 8/30
172/172 [==============================] - 13s 62ms/step - loss: 1.3079
Epoch 9/30
172/172 [==============================] - 12s 62ms/step - loss: 1.2732
Epoch 10/30
172/172 [==============================] - 12s 61ms/step - loss: 1.2392
Epoch 11/30
172/172 [==============================] - 12s 61ms/step - loss: 1.2063
Epoch 12/30
172/172 [==============================] - 12s 61ms/step - loss: 1.1750
E

###Load the Model
We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one piece of text to the model and have it make a prediction.

In [22]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model finishes training, we can find the **lastest checkpoint** that stores the models weights using the following line.

In [23]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

We can load **any checkpoint** we want by specifying the exact file to load.

In [24]:
# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

###Generate Text
Generate some text using any starting string we choose.

Generating text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.  Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.

![alt text](https://www.tensorflow.org/text/tutorials/images/text_generation_sampling.png)


In [25]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [28]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: The meaning of life
The meaning of life, boy, to have him had,
The preving fortune of the prince, and have speed to him.

BUSHY:
Despission, the carpetsied with a flowering face!
Why stay become on himself? I have out his heir behold me, when you perpetuate these sighs from thee sorrow,
they will not.

POLIXENES:
Say'st thou, pray to,
And yet we her suit me as dear as London's assurance;
Thou call'dst me free better;
And, in those war will prove him,
Had I alreedy leaden he only. What, was Quke me twhile to age, depart to heaven,
In her I see, hath powd.

JULIET:
I' the mother of that side, I look'd on Death.

TRANIO:
Fear not, my lord, let us amointed, cloid
Of any body: but a ran that till he been to
Thanks, stand back again, if something of their sovereigns, they are ins their uncleased from the champed;
of their bows, se up


##Sources

1. Chollet François. Deep Learning with Python. Manning Publications Co., 2018.
2. “Text Generation with an RNN &nbsp;: &nbsp; TensorFlow Core.” TensorFlow, www.tensorflow.org/tutorials/text/text_generation.
3. “Understanding LSTM Networks.” Understanding LSTM Networks -- Colah's Blog, https://colah.github.io/posts/2015-08-Understanding-LSTMs/.